First draft based on https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

Needs to be cleaned up and improved :)

In [0]:
!pip install pytorch_pretrained_bert

In [0]:
!pip install seqeval

In [0]:
import torch
import urllib
import pandas as pd
from pytorch_pretrained_bert import BertForTokenClassification, BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
MAX_LEN = 50
batch_size = 32

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.cuda()

In [0]:
TRAIN_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/train-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AD7GEDK3MI27HVJPQWOE74C6FBZHA'
DEV_URL = 'https://raw.githubusercontent.com/cicl-iscl/CyberWallE/master/data/dev-improved-sentiwordnet-arguingfullindiv-pos.tsv?token=AD7GEDM3LOMZM6MP4HZS4MS6FBZHK'

In [0]:
def get_comments(filename, url=True):
    if url:
        comments = []
        with urllib.request.urlopen(filename) as f:
            for line in f:
                if line.startswith(b'#'):
                    comments.append(line.decode("utf-8"))
                else:
                    break
        return comments
    with open(filename, 'r', encoding='utf8') as f:
        commentiter = takewhile(lambda s: s.startswith('#'), f)
        comments = list(commentiter)
    return comments

In [0]:
comments = get_comments(TRAIN_URL)
train_df = pd.read_csv(TRAIN_URL, sep='\t', skiprows=len(comments), quoting=3)
train_input = train_df.groupby('sent_id')['token'].apply(list).to_frame()

In [0]:
sentences = train_input['token'].tolist()
sentences = [['[CLS]'] + [str(word).lower() for word in sent] + ['[SEP]'] for sent in sentences]
sentences[0]

In [0]:
labels = train_df.groupby('sent_id')['label'].apply(list).to_frame()['label'].tolist()
labels[0]

In [0]:
tokenized_texts = [tokenizer.tokenize(' '.join(sent)) for sent in sentences]
print(tokenized_texts[0])
print(tokenized_texts[8])

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
tag2idx = {'O': 0, 'B': 1, 'I': 1}
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

In [0]:
tags_vals = ['O', 'I']

In [0]:
from tqdm import trange
epochs = 3
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print(pred_tags)
    print(valid_tags)

In [0]:
comments = get_comments(DEV_URL)
dev_df = pd.read_csv(DEV_URL, sep='\t', skiprows=len(comments), quoting=3)
dev_input = dev_df.groupby('sent_id')['token'].apply(list).to_frame()
sentences = dev_input['token'].tolist()
sentences = [['[CLS]'] + [str(word).lower() for word in sent] + ['[SEP]'] for sent in sentences]
sentences[0]

In [0]:
tokenized_texts = [tokenizer.tokenize(' '.join(sent)) for sent in sentences]
print(tokenized_texts[0])
print(tokenized_texts[8])

In [0]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
pred_inputs = torch.tensor(input_ids)
pred_masks = torch.tensor(attention_masks)

pred_data = TensorDataset(pred_inputs, pred_masks)
pred_sampler = SequentialSampler(pred_data)
pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=batch_size)

In [0]:
model.eval()

In [0]:
predictions = []

# Predict 
for batch in pred_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  
  # Store predictions and true labels
#   predictions.append(logits)
  predictions.extend([list(p) for p in np.argmax(logits, axis=2)])

In [0]:
preds = []
idx2label = {0: 'O', 1: 'I'}
for mask, pred in zip(attention_masks, predictions):
    sent_pred = []
    for m, p in zip(mask, pred):
        if m > 0.0:
            sent_pred.append(idx2label[p])
    preds.append(sent_pred)

In [0]:
dev_input['n_toks'] = dev_input['token'].apply(lambda x: len(x))
dev_input.head()

In [0]:
len(tokenized_texts)

In [0]:
preds_with_toolong = []
for row in dev_input.itertuples():
    sent_idx = row.Index - 1
    sent_pred = preds[sent_idx]
    n_actual_toks = row.n_toks
    n_pred_toks = len(sent_pred)

    pred_without_subwordtoks = []
    for tok, pred in zip(tokenized_texts[sent_idx], sent_pred):
        if tok == '[CLS]' or tok == '[SEP]':
            continue
        if tok.startswith('##'):
            pred_without_subwordtoks[-1] = pred
        else:
            pred_without_subwordtoks.append(pred)

    for _ in range(n_actual_toks - len(pred_without_subwordtoks)):
        pred_without_subwordtoks.append('O')
    preds_with_toolong.append(pred_without_subwordtoks)


In [0]:
len(preds_with_toolong)

In [0]:
flat_predictions = [item for sublist in preds_with_toolong for item in sublist]
len(flat_predictions)

In [0]:
dev_df.shape

In [0]:
dev_df["label"] = flat_predictions
dev_df.head()
dev_df.to_csv("dev-task-SI.out", sep='\t', header=True, index=False)

In [0]:
def si_predictions_to_spans(label_df):
    spans = []
    prev_label = 'O'
    prev_span_start = '-1'
    prev_span_end = '-1'
    prev_article = ''

    for row in label_df.itertuples():
        article = row.document_id
        span_start = row.token_start
        span_end = row.token_end
        label = row.label

        span, prev_span_start = update_prediction(article, label,
                                                  span_start, span_end,
                                                  prev_article, prev_label,
                                                  prev_span_start,
                                                  prev_span_end)
        if span is not None:
            spans.append(span)

        prev_article = article
        prev_label = label
        prev_span_end = span_end

    # Make sure we get the last prediction
    span, _ = update_prediction(article, label, span_start, span_end,
                                prev_article, prev_label, prev_span_start,
                                prev_span_end)
    if span is not None:
        spans.append(span)
    return spans

# Helper method for si_predictions_to_spans
def update_prediction(article, label, span_start, span_end, prev_article,
                      prev_label, prev_span_start, prev_span_end):
    span = None
    cur_span_start = prev_span_start
    # Ending a span: I-O, B-O, I-B, B-B, new article
    if prev_label != 'O' and (label != 'I' or prev_article != article):
        span = (prev_article, prev_span_start, prev_span_end)

    # Starting a new span: O-B, O-I, I-B, B-B, new article
    if label == 'B' or (label == 'I' and prev_label == 'O') \
            or prev_article != article:
        # Update the start of the current label span
        cur_span_start = span_start
    return span, cur_span_start


def print_spans(spans, file_prefix, file_stem, file_suffix):
    outfile = file_prefix + 'spans_' + file_stem + '_' + file_suffix + '.txt'
    with open(outfile, mode='w') as f:
        for span in spans:
            f.write(str(span[0]) + '\t' + str(span[1]) + '\t' +
                    str(span[2]) + '\n')

In [0]:
spans = si_predictions_to_spans(dev_df)
print_spans(spans, '', 'bert', '')